In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.utils import plot_model
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.regularizers import L2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create the Model


In [ ]:
from tensorflow.keras.applications import EfficientNetB4, DenseNet121

# USE EFFICIENT-NET
# pre_trained_model_256 = EfficientNetB4(include_top = False, input_shape = (256, 256, 3))
# pre_trained_model_128 = EfficientNetB4(include_top = False, input_shape = (128, 256, 3))

# USE DENSENET121
pre_trained_model_256 = DenseNet121(include_top = False, input_shape = (256,256,3), weights = 'imagenet', pooling = 'avg') #, pooling = 'avg'
pre_trained_model_256_0 = DenseNet121(include_top = False, input_shape = (256,256,3), weights = 'imagenet', pooling = 'avg') #, pooling = 'avg'
pre_trained_model_128 = DenseNet121(include_top = False, input_shape = (128,256,3), weights = 'imagenet', pooling = 'avg') #, pooling = 'avg'

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
pre_trained_model_128.trainable = False
pre_trained_model_256.trainable = False
pre_trained_model_256_0.trainable = False

for layer in pre_trained_model_128.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
for layer in pre_trained_model_256.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
for layer in pre_trained_model_256_0.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

In [ ]:
last_layer_256 = pre_trained_model_256.get_layer('avg_pool')  # avg_pool
last_output_256 = last_layer_256.output

last_layer_256_0 = pre_trained_model_256_0.get_layer('avg_pool')  # avg_pool
last_output_256_0 = last_layer_256_0.output

last_layer_128 = pre_trained_model_128.get_layer('avg_pool')
last_output_128 = last_layer_128.output

In [ ]:
def initialize_base_model(last_output, pre_trained_model, name):
    x = layers.Dropout(0.3)(last_output)
    x = layers.Dense(1024, activation = 'relu', kernel_regularizer = L2())(x) 
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation = 'relu', kernel_regularizer = L2())(x)

    return Model(pre_trained_model.input, x, name = name)

In [ ]:
num_of_classes = 5

model_1 = initialize_base_model(last_output_256, pre_trained_model_256, name = 'DenseNet_Ax1')
model_2 = initialize_base_model(last_output_256_0, pre_trained_model_256_0, name = 'DenseNet_Ax2')

input_1 = tf.keras.Input(shape = (256, 256, 3), name= 'input_axis_1')
output_1 = model_1(input_1)

input_2 = tf.keras.Input(shape = (256, 256, 3), name= 'input_axis_2')
output_2 = model_2(input_2)

x = layers.Concatenate()([output_1, output_2])
x = layers.Dropout(0.2)(x)
output = layers.Dense(num_of_classes, activation = 'softmax')(x)

model = Model(inputs = [input_1, input_2], outputs = output)

model.compile(optimizer = Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = [tfa.metrics.CohenKappa(num_classes = num_of_classes), 'accuracy'])



In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

# Image Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
base_path = 'gdrive/MyDrive/ipre_dataset'

df = pd.read_csv(f'{base_path}/labels.csv')
labels = df['TypeOfTB'] - 1

In [ ]:
def double_generator(generator1, generator2):

    while True:
        for (x1,y1),(x2,y2) in zip(generator1, generator2):
            yield ([x1,x2],y1)

In [ ]:
threshold = 0.18
experiment_name = 'Best_2D_Approach'

class CK_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if logs.get('val_cohen_kappa') > threshold and logs.get('cohen_kappa') > threshold:
            print('Training And Validation Cohens Kappa achieved')
            self.model.stop_training = True

cohens_kappa_callback = CK_callback()

logdir = os.path.join("logs", experiment_name)
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
#                               patience=5, min_lr=1e-7)

In [ ]:

base_path = 'gdrive/MyDrive/ipre_dataset/ordered'

train_dir_ax0 = f'{base_path}/axis_0_ord/train'
validation_dir_ax0 = f'{base_path}/axis_0_ord/validation'

train_dir_ax1 = f'{base_path}/axis_1_ord/train'
validation_dir_ax1 = f'{base_path}/axis_1_ord/validation'

train_dir_ax2 = f'{base_path}/axis_2_ord/train'
validation_dir_ax2 = f'{base_path}/axis_2_ord/validation'

batch_size = 32

train_datagen_ax0 = ImageDataGenerator(rescale=1/255,
                                       rotation_range = 25,
                                       width_shift_range = 0.15,
                                       height_shift_range = 0.15,
                                       zoom_range = 0.2,
                                       horizontal_flip = True,
                                       vertical_flip = True)

train_datagen_ax1 = ImageDataGenerator(rescale=1/255,
                                       rotation_range = 25,
                                       width_shift_range = 0.15,
                                       height_shift_range = 0.15,
                                       zoom_range = 0.2,
                                       horizontal_flip = True,
                                       vertical_flip = True)

train_datagen_ax2 = ImageDataGenerator(rescale=1/255,
                                       rotation_range = 25,
                                       width_shift_range = 0.15,
                                       height_shift_range = 0.15,
                                       zoom_range = 0.2,
                                       horizontal_flip = True,
                                       vertical_flip = True)


train_generator_ax0 = train_datagen_ax0.flow_from_directory(train_dir_ax0, batch_size = batch_size, class_mode = 'categorical', target_size = (128, 256))
train_generator_ax1 = train_datagen_ax1.flow_from_directory(train_dir_ax1, batch_size = batch_size, class_mode = 'categorical', target_size = (256, 256))
train_generator_ax2 = train_datagen_ax2.flow_from_directory(train_dir_ax2, batch_size = batch_size, class_mode = 'categorical', target_size = (256, 256))

validation_datagen_ax0 = ImageDataGenerator(rescale=1/255)
validation_datagen_ax1 = ImageDataGenerator(rescale=1/255)
validation_datagen_ax2 = ImageDataGenerator(rescale=1/255)


validation_generator_ax0 = validation_datagen_ax0.flow_from_directory(validation_dir_ax0, batch_size = batch_size, class_mode = 'categorical', target_size = (128, 256))
validation_generator_ax1 = validation_datagen_ax1.flow_from_directory(validation_dir_ax1, batch_size = batch_size, class_mode = 'categorical', target_size = (256, 256))
validation_generator_ax2 = validation_datagen_ax2.flow_from_directory(validation_dir_ax2, batch_size = batch_size, class_mode = 'categorical', target_size = (256, 256))



In [ ]:
history = model.fit(
    double_generator(train_generator_ax1, train_generator_ax2), 
    validation_data = double_generator(validation_generator_ax1, validation_generator_ax2),
    steps_per_epoch = 40,
    epochs = 100,
    validation_steps = 18,
    callbacks = [tensorboard_callback, cohens_kappa_callback],
    class_weight = classes_weight
)


**Save the model**

In [ ]:
model_json = model.to_json()
path = 'gdrive/MyDrive/ipre_dataset/saved_models/Best_2D_Approach/'
with open(f"{path}model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights(f"{path}model_num.h5")